In [13]:
import pandas as pd
import numpy as np
import os as os
#import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import classification_report
from joblib import dump, load

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
df = pd.read_csv('sentimento - base3.csv')

In [15]:
df.head()

,id_issue,mes_chat,SENTIMENTO,TIPO_MSG,hierarquia_4,ds_cluster_sociodemographic,ds_message,flag_negativo_indecx
0,15388962,2023-04-01,neutro,duvida,Vencimento fatura,"0 - Mulher, Casada, 25 anos ou mais, cidade Pe...","['Que dia minha fatura fecha?','Ainda tenho dú...",0
1,17497395,2023-07-01,neutro,duvida,Dúvidas sobre pagamento de fatura,"11 - Homem, Solteiro, Até 24 anos, cidade Pequ...","['Quero pagar os 100 reais que eu tirei','Mais...",0
2,15213047,2023-04-01,neutro,solicitacao,Não reconhecimento de compra no crédito,"17 - Mulher, 25 a 40 anos, Sudeste e Sul","['Apareceu uma compra que eu não fiz','Dúvida ...",0
3,17117301,2023-06-01,neutro,problema,Parcelamento de fatura,"8 - Mulher, Solteira, Até 24 anos, cidade Pequ...",['No meu limite está escrito que está utilizad...,0
4,17345996,2023-07-01,negativo,problema,Dúvidas PIX,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...","['Pix','E merda o will nunca teve essas coisas...",0


In [16]:
len(df)

14906

In [17]:
exemplo_base = df[df['SENTIMENTO'].notnull()]
len(exemplo_base)

645

In [18]:
stopwords_will= ['da', 'em', 'você', 'de', 'ao', 'os', 'que', 'e', 'eu', 'o', 'estou', 'ele','ela','pra', 'pro', 'entao', 'para', 'voce',
                 'pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro',
            'seu', 'sua', 'por', 'sobre', 'mais', 'uma', 'um','como', 'meu', 'com', 'outro','outra','das','dos','foi', 'fiz','pelo','tem','mas',
            'este', 'esse','ta', 'to', 'ai', 'la', 'lo', 'lhe', 'ne','pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro', 'isso']

### Modelo Sentimento

In [19]:
exemplo_base['SENTIMENTO_label'] = np.where(exemplo_base['SENTIMENTO'] == "negativo", 0, 
                                            np.where(exemplo_base['SENTIMENTO'] == "neutro", 1, 2 ))

/tmp/ipykernel_977523/2720231541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exemplo_base['SENTIMENTO_label'] = np.where(exemplo_base['SENTIMENTO'] == "negativo", 0,


In [20]:
exemplo_base['SENTIMENTO_label'].value_counts()

0    265
1    258
2    122
Name: SENTIMENTO_label, dtype: int64

In [21]:
from sklearn.neural_network import MLPClassifier

vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message'])
y = exemplo_base.SENTIMENTO_label


x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

In [22]:
# Study OPTUNA: {'hidden_layer_sizes': 130, 'activation': 'relu', 'alpha': 0.18641747634500205}
clf_sentiment = MLPClassifier(solver='lbfgs', alpha=0.18641747634500205, hidden_layer_sizes= 130, activation = 'relu', warm_start = True)
clf_sentiment.fit(x_train, y_train)

MLPClassifier(alpha=0.18641747634500205, hidden_layer_sizes=130, solver='lbfgs',
              warm_start=True)

In [23]:
y_predicted = clf_sentiment.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,43,4,6
1,14,41,5
2,1,2,13


In [24]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.81      0.74      0.77        58
           1       0.68      0.87      0.77        47
           2       0.81      0.54      0.65        24

    accuracy                           0.75       129
   macro avg       0.77      0.72      0.73       129
weighted avg       0.76      0.75      0.75       129



In [25]:
dump(clf_sentiment, 'modelo_sentimento.joblib') 

['modelo_sentimento.joblib']

In [26]:
dump(vetorizador, 'vetorizador_sentimento.joblib') 

['vetorizador_sentimento.joblib']

In [27]:
# No Sagemaker
#clf = load('filename.joblib') 

### Avaliando o acerto com as palavras chave

In [28]:
base_predita = exemplo_base[['ds_message', 'SENTIMENTO_label']].copy()
base_predita['sent_predito_1'] = clf_sentiment.predict(X)

In [29]:
base_predita.head()

,ds_message,SENTIMENTO_label,sent_predito_1
0,"['Que dia minha fatura fecha?','Ainda tenho dú...",1,1
1,"['Quero pagar os 100 reais que eu tirei','Mais...",1,1
2,"['Apareceu uma compra que eu não fiz','Dúvida ...",1,1
3,['No meu limite está escrito que está utilizad...,1,1
4,"['Pix','E merda o will nunca teve essas coisas...",0,0


### Palavras chave 

In [30]:
chave_negativo = ['falta de respeito', 
'judicial',
'procon',
'jurídico',
'porra',
'merda', 
'palhaçada',
'carai',
'caralho',
'bosta',
'vocês demoram',
'vcs demoram',
'horrivel',
'banco ruim',
'péssimo',
'danos morais',
'pelo amor de deus',
'eu preciso resolver',
'me roubando']


In [31]:
chave_positivo = [
'ótimo',
'perfeito',
'muito obrigada',
'excelente',
'ótimo dia',
'maravilha',
'deus abençoe']

In [32]:
base_predita['sent_predito_1'].value_counts()

1    271
0    260
2    114
Name: sent_predito_1, dtype: int64

In [33]:
def marca_negativo(x):
    for i in chave_negativo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
base_predita['flag_negativo'] = base_predita['ds_message'].apply(marca_negativo)

In [34]:
def marca_positivo(x):
    for i in chave_positivo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
base_predita['flag_positivo'] = base_predita['ds_message'].apply(marca_positivo)

In [35]:
base_predita['sent_predito_2'] = np.where(base_predita['flag_negativo'] == 1,  0, #condições negativo
                                 np.where(base_predita['flag_positivo'] == 1,  2, #condições positivo
                                          base_predita['sent_predito_1']))

base_predita['sent_predito_2'].value_counts()

1    270
0    259
2    116
Name: sent_predito_2, dtype: int64

In [36]:
print(classification_report(base_predita['SENTIMENTO_label'], base_predita['sent_predito_1']))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       265
           1       0.93      0.98      0.95       258
           2       0.97      0.91      0.94       122

    accuracy                           0.95       645
   macro avg       0.96      0.94      0.95       645
weighted avg       0.95      0.95      0.95       645



In [37]:
print(classification_report(base_predita['SENTIMENTO_label'], base_predita['sent_predito_2']))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       265
           1       0.93      0.98      0.95       258
           2       0.96      0.91      0.93       122

    accuracy                           0.95       645
   macro avg       0.95      0.94      0.95       645
weighted avg       0.95      0.95      0.95       645



Como o precision e recall não melhoraram vou manter apenas o modelo

### Modelo do tipo de mensagem

In [38]:
exemplo_base['TIPO_MSG'].value_counts()

problema       303
duvida         194
solicitacao    148
Name: TIPO_MSG, dtype: int64

In [39]:
exemplo_base['TIPO_MSG_label'] = np.where(exemplo_base['TIPO_MSG'] == "problema", 0, 
                                            np.where(exemplo_base['TIPO_MSG'] == "solicitacao", 1, 2 ))

/tmp/ipykernel_977523/387629369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exemplo_base['TIPO_MSG_label'] = np.where(exemplo_base['TIPO_MSG'] == "problema", 0,


In [40]:
exemplo_base['TIPO_MSG_label'].value_counts()

0    303
2    194
1    148
Name: TIPO_MSG_label, dtype: int64

In [41]:

vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message'])
y = exemplo_base.TIPO_MSG_label

x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.TIPO_MSG_label
                                                    , random_state=42, test_size=0.20)

In [42]:
from sklearn.neural_network import MLPClassifier
clf_tipo_msg = MLPClassifier(solver='lbfgs', alpha=3.4372969952170234, hidden_layer_sizes= 160, activation = 'tanh', warm_start = True)
clf_tipo_msg.fit(x_train, y_train)

y_predicted = clf_tipo_msg.predict(x_test)
pd.crosstab(y_predicted, y_test)

TIPO_MSG_label,0,1,2
row_0,,,
0,54,13,11
1,4,12,2
2,1,7,25


In [43]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.69      0.92      0.79        59
           1       0.67      0.38      0.48        32
           2       0.76      0.66      0.70        38

    accuracy                           0.71       129
   macro avg       0.71      0.65      0.66       129
weighted avg       0.71      0.71      0.69       129



In [44]:
dump(clf_tipo_msg, 'modelo_tipo_mensagem.joblib')

['modelo_tipo_mensagem.joblib']